In [2]:
import pandas as pd
import numpy as np

# Put your own path 
df = pd.read_csv(r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv')

print(df.columns.tolist())

['ticker', 'open', 'high', 'low', 'close', 'volume', 'OPCL', 'pvCLCL', 'prevAdjClose', 'SPpvCLCL', 'sharesOut', 'PERMNO', 'SICCD', 'PERMCO', 'prevRawOpen', 'prevRawClose', 'prevAdjOpen']


We first prepare the dataframe we will use to construct the portfolio: df_cleaned which contains the 5531 returns of 632 assets

In [3]:
import pandas as pd
import ast

# Function to safely convert a string into a list
def safe_literal_eval(s):
    try:
        # Tries to convert the string into a list
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # If an error occurs, returns a default value, e.g. an empty list
        return []

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(safe_literal_eval)
df['close'] = df['close'].apply(safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)



In [5]:
new_df = df[['ticker', 'return']] # creat a new data frame with the column ticker and return 

In [6]:
# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

def check_nan_inf(df):
    # Vérification des valeurs NaN
    if df.isna().any().any():
        print("Il y a des valeurs NaN dans le DataFrame")
    else:
        print("Il n'y a pas de valeurs NaN dans le DataFrame")

def remove_rows_with_nan(df):
    return df.dropna()

df_cleaned = remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

check_nan_inf(df_cleaned)

df_cleaned.shape

Il n'y a pas de valeurs NaN dans le DataFrame


(632, 5532)

Now we get a clustering with SPONGE algorithm

In [81]:
import sys

sys.path.append('C:/Users/33640/OneDrive/Documents/GitHub/Portfolio_clustering_project')  # Ajoute le chemin parent

from signet.cluster import Cluster 
from scipy import sparse
def signed_adjency(mat):
    '''
    L'idée est ici, à partir d'une matrice de corrélation mat, de renvoyer deux matrices 
    A_positive et A_negative qui correspondraient aux matrices des corrélations positives et négatives 
    associées  
    '''

    A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
    A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
    
    return A_pos, A_neg
def apply_SPONGE(correlation_matrix, k): 

    '''
    IDÉE : étant donné une matrice de correlation obtenue à partir d'une base de donnée et de la similarité de pearson, renvoyer un vecteur associant 
           à chaque actif le numéro du cluster auquel il appartient une fois qu'on lui a appliqué SPONGE (à partir du package signet)

    PARAMS : 

    - correlation_matrix : a square dataframe of size (number_of_stocks, number_of_stocks)
    - k : the number of clusters to identify. If a list is given, the output is a corresponding list

    RETURNS : array of int, or list of array of int: Output assignment to clusters.

    '''
    
    ## On respecte le format imposé par signet. Pour cela il faut changer le type des matrices A_pos et A_neg, qui ne peuvent pas rester des dataframes 

    A_pos, A_neg = signed_adjency(correlation_matrix)

    A_pos_sparse = sparse.csc_matrix(A_pos.values)
    A_neg_sparse = sparse.csc_matrix(A_neg.values)

    data = (A_pos_sparse, A_neg_sparse)

    cluster = Cluster(data)

    ## On applique la méthode SPONGE : clusters the graph using the Signed Positive Over Negative Generalised Eigenproblem (SPONGE) clustering.

    return cluster.SPONGE(k )

we divide the timeframe in 28 periods of 200 days (the last one is 132 days only) and run SPONGE to make 28 clusterings

In [104]:

data_period=[]
result=[]
for i in range(28):
    start_col = i * 200 + 1  # Commence à partir de la deuxième colonne
    end_col = start_col + 200

    # Vérifie si la fin de la tranche dépasse le nombre de colonnes
    if end_col > len(df_cleaned.columns):
        end_col = len(df_cleaned.columns)  # Ajuste pour ne pas dépasser
    
    data_period.append(df_cleaned.iloc[:, start_col:end_col]) 

    returns_transposed = data_period[i].transpose()

    # Calculer la matrice de corrélation sur les actifs transposés
    correlation_matrix = returns_transposed.corr(method='pearson')
    result.append(apply_SPONGE(correlation_matrix, k=9))



Now we use  hierarchical clustering to construct a clustering from the 28 clusterings we made

In [105]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

# Supposons que vous avez une liste de listes, où chaque sous-liste est un des 28 clusterings
# Chaque sous-liste contient 632 éléments avec des valeurs de 0 à 9
clusterings = result  # Remplacez ceci par vos données de clustering

# Convertir les résultats de clustering en une matrice de co-occurrence
n_data = 632
co_occurrence_matrix = np.zeros((n_data, n_data))

for clustering in clusterings:
    for i in range(n_data):
        for j in range(n_data):
            if clustering[i] == clustering[j]:
                co_occurrence_matrix[i, j] += 1

# Convertir en matrice de similarité
similarity_matrix = co_occurrence_matrix / len(clusterings)

# Convertir la matrice de similarité en matrice de distance
distance_matrix = 1 - similarity_matrix

# Convertir la matrice de distance en format condensé pour la fonction linkage
# car linkage attend une matrice de distance condensée pour les données non euclidiennes
tri_upper_idx = np.triu_indices(n_data, k=1)
distance_condensed = distance_matrix[tri_upper_idx]

# Effectuer le clustering hiérarchique
Z = linkage(squareform(distance_condensed), method='average')

# Former des clusters
# Vous pouvez ajuster le 't' pour obtenir le nombre de clusters souhaité
clusters = fcluster(Z, t=9, criterion='maxclust')
clusters=clusters-1
# Afficher les clusters
print(clusters)

[0 0 1 0 1 0 4 0 8 4 2 0 0 0 0 0 5 1 0 0 0 0 0 0 0 2 0 0 0 6 0 0 5 0 8 0 2
 0 5 0 8 5 5 0 0 7 5 0 0 1 0 0 0 0 1 0 0 0 5 1 0 1 5 0 0 0 0 5 3 3 0 0 0 1
 0 8 0 0 0 1 1 0 0 5 0 4 0 0 0 0 8 0 4 0 0 1 0 0 7 7 4 6 2 0 4 0 0 4 3 0 0
 6 0 2 0 4 6 5 5 0 0 6 0 2 0 0 0 5 1 0 5 0 3 4 2 0 0 0 0 7 0 0 7 0 0 7 0 0
 5 0 6 3 8 4 4 6 0 0 0 0 4 0 5 4 1 0 0 0 0 0 6 6 5 5 6 0 5 0 4 7 0 0 0 0 0
 0 0 0 2 0 2 2 2 0 0 4 0 0 0 1 0 0 5 5 0 8 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7
 4 0 0 0 0 0 0 8 0 0 2 6 0 5 0 4 0 0 7 5 7 8 0 0 0 0 6 0 0 5 0 4 0 4 1 0 0
 7 0 4 0 0 2 0 0 0 2 6 0 0 3 2 0 0 0 0 0 1 0 0 0 0 0 4 0 0 0 0 5 4 0 0 4 2
 1 5 3 0 3 2 0 0 3 2 1 0 1 0 0 0 4 0 0 5 2 0 5 0 5 5 0 0 0 0 0 3 2 1 7 0 0
 0 1 5 0 3 3 0 3 3 0 3 3 3 4 0 0 0 0 7 3 4 0 3 3 3 1 6 7 0 0 0 0 0 0 0 0 3
 3 3 6 3 3 0 3 3 0 3 3 3 3 3 6 0 3 0 8 6 5 4 5 0 0 5 5 0 0 0 6 0 7 0 0 3 1
 0 0 3 0 5 2 0 0 4 5 0 3 6 6 0 0 0 0 6 0 8 4 4 5 6 4 7 1 7 4 1 0 0 2 0 0 7
 0 0 0 5 3 3 0 4 0 4 0 4 7 5 5 0 0 6 0 2 0 7 0 0 5 0 0 0 0 6 0 0 2 2 0 0 0
 2 0 0 6 0 0 0 0 0 0 2 8 

C:\Users\33640\AppData\Local\Temp/ipykernel_25820/2933028627.py:31: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(squareform(distance_condensed), method='average')


Now we want to run markowitz on our portfolio, we need to get the returns and the covariance matrix of the returns of each asset, but before we need to get the composition of each cluster which is a combination of multiple assets. 

In [106]:
cluster_comp=[[],[],[],[],[],[],[],[],[]]
cluster_comp2=[]
for c in range(632):
    cluster_comp[clusters[c]].append(df_cleaned.iloc[c, 0])
for cluster in cluster_comp:
    cluster_indices = [df_cleaned.index[df_cleaned['ticker'] == element].tolist() for element in cluster]
    cluster_comp2.append([index for sublist in cluster_indices for index in sublist])



print(cluster_comp2[3])

[68, 69, 108, 132, 151, 272, 298, 300, 304, 327, 337, 338, 340, 341, 343, 344, 345, 352, 355, 356, 357, 369, 370, 371, 373, 374, 376, 377, 379, 380, 381, 382, 383, 386, 405, 409, 418, 448, 449, 585, 587, 591]


Now we compute the intracluster weights with gaussianweight

In [107]:

# Fonction pour calculer la distance gaussienne
def gaussian_distance(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * sigma**2))


# Paramètre sigma pour la distance gaussienne
sigma = 1.0  # vous pouvez ajuster cette valeur

# Calculer les poids pour chaque élément dans chaque cluster
cluster_weights = []
for cluster in cluster_comp2:
    cluster_data = df_without_first_column.iloc[cluster, :]
    cluster_center = cluster_data.mean()  # Calcul du centre du cluster
    if isinstance(cluster_data, pd.Series):
        cluster_data = pd.DataFrame([cluster_data])
    weights = np.array([np.exp(- 2*(gaussian_distance(row, cluster_center, sigma)**2)) for _, row in cluster_data.iterrows()])
    weights_normalized = weights / weights.sum()
    cluster_weights.append(weights_normalized)

In [108]:
normalized_weights=[0 for i in range(632)]
for i in range(len(cluster_comp2)):
    for j in range(len(cluster_comp2[i])):
        normalized_weights[cluster_comp2[i][j]]=cluster_weights[i][j]

print(normalized_weights)

[0.00342803346626694, 0.0027003325337977137, 0.024925983147876613, 0.003430287296109731, 0.03760080276408378, 0.0010265756490040838, 0.016869428660835444, 0.0018268497260530935, 0.07165328426391877, 0.021620976204285623, 0.03242293646691621, 0.0023666535669290565, 0.0025489996670799266, 0.004335329375009703, 0.0030412777794537327, 0.004101761640541757, 0.0190881519086567, 0.033088466664930975, 0.002694799277105192, 0.0037703298255496024, 0.002103114286262196, 0.0018792229011134656, 0.004353588279805855, 0.0018534965239900502, 0.0028442606924653745, 0.031126461917735698, 0.0032505420345885546, 0.004023369797315107, 0.002536511374164698, 0.022392000184836093, 0.0018316797922898552, 0.0025901537444293363, 0.012483841006891346, 0.002750559743771264, 0.04595203361268621, 0.0020273619893994574, 0.03306195699511693, 0.0024763817543921323, 0.019722187129342126, 0.0019333981616588428, 0.0682284995085667, 0.028018474079050008, 0.011708798593968462, 0.003017888442775482, 0.0018654726930022304, 0.

We get the means of the returns of each cluster, and the covariance of the returns of the clusters.

In [109]:
return_clusters = pd.DataFrame(columns=df_cleaned.columns[1:])

# Remplissage du nouveau DataFrame avec les moyennes des colonnes pour chaque cluster
for k in range(9):
    # Sélection des lignes appartenant au cluster k
    rows_in_cluster = df_cleaned.iloc[cluster_comp2[k]]

    # Calcul de la moyenne pour chaque colonne, en excluant la première colonne
    mean_values = (rows_in_cluster.iloc[:, 1:].mul(cluster_weights[k], axis=0)).sum()

    # Ajout des moyennes calculées au nouveau DataFrame
    return_clusters.loc[k] = mean_values

print(return_clusters.shape)
returns_transposed = return_clusters.transpose()

# Calculer la matrice de corrélation sur les actifs transposés
correlation_matrix = returns_transposed.cov()*np.sqrt(252)


annual_expected_returns=(return_clusters + 1).prod(axis=1)-1

# Afficher la matrice de corrélation
print(correlation_matrix)
print(annual_expected_returns)

(9, 5531)
          0         1         2         3         4         5         6  \
0  0.002496  0.001268  0.002016  0.000240  0.000928  0.002145  0.002363   
1  0.001268  0.001172  0.001083  0.000154  0.000818  0.001144  0.001176   
2  0.002016  0.001083  0.002048  0.000206  0.000838  0.001780  0.002157   
3  0.000240  0.000154  0.000206  0.000405  0.000156  0.000279  0.000289   
4  0.000928  0.000818  0.000838  0.000156  0.001199  0.001096  0.000944   
5  0.002145  0.001144  0.001780  0.000279  0.001096  0.003091  0.001981   
6  0.002363  0.001176  0.002157  0.000289  0.000944  0.001981  0.005126   
7  0.000701  0.000384  0.000596  0.000320  0.000324  0.000727  0.000809   
8  0.001086  0.000557  0.001059  0.000273  0.000510  0.001030  0.001974   

          7         8  
0  0.000701  0.001086  
1  0.000384  0.000557  
2  0.000596  0.001059  
3  0.000320  0.000273  
4  0.000324  0.000510  
5  0.000727  0.001030  
6  0.000809  0.001974  
7  0.000848  0.000571  
8  0.000571  0.002699  

We run Markowitz

In [110]:
from module1 import markowitz

weights=markowitz(annual_expected_returns, correlation_matrix)

print(weights)

OrderedDict([(0, 0.0), (1, 0.58907), (2, 0.0), (3, 0.0), (4, 0.41093), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)])


Our final cumulated(daily) PnL and our Sharpe Ratio 

In [111]:
from module1 import portfolio_pnl_sharpe
portfolio_pnl_sharpe(returns_transposed, weights, risk_free_rate=0.03)

(return_0        0.985690
 return_1        0.984650
 return_2        1.001461
 return_3        1.015362
 return_4        1.038689
                  ...    
 return_5526    10.377445
 return_5527    10.430285
 return_5528    10.457851
 return_5529    10.436657
 return_5530    10.458868
 Length: 5531, dtype: float64,
 0.6724493231615052)